In [1]:
import yaml
import numpy as np
import collections
import string
import copy

In [2]:
import pandas as pd
from yaml import safe_load
from pandas import json_normalize
from difflib import SequenceMatcher

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
import spacy

In [5]:
import glob
import re

In [6]:
data_path = "../data/eb07-v1.2-TXT/"

count = 0
#num pages == num of files in each folder
parsed_data = []

for file in sorted(glob.glob(data_path + "*/*.txt")):
    count += 1
    if count > 30:
        break
    
    with open (file, 'r') as currFile:
        text = currFile.read()
        #print(text)
        text = text.split("=+")
        
        #get data of location in EB
        ed_info = re.findall("\[\d+\:\d+\:\d+\]", text[1])[0].replace("[", "").replace("]", "")
        ed_info = ed_info.split(":")
        ed = ed_info[0]
        vol = ed_info[1]
        page = ed_info[2]
        
        
        
        
        
        #Name adjkfadsjhlasdgjhasgdlj
        #Name can also as adsjklgadsl
        
        #Split by terms
        #TERM, def 
        #Term, def - same
        #TERM, def - different
        #TERM TERM def
        #TERM def
        # TERM, def
        # def
        #TERM, def
        #def dfsakadsfh [#:##:##] adfjlfahkd
        #AA, def
        #Aa, def
        #Aa, def
        #A, 
        
        #make DF then debug
        
        #TODO: remove "   ,   " from beginning
        #TODO: NAPOLEON'S LAND is split into NAPOLEON and 'S
        
        lines = text[2].split('\n')
        lines = [x for x in lines if x]
        
        capital_pattern = '^[A-Z\s\']+(?![a-z])'
        
        term = ""
        definition = ""
        for line in lines:
            #print(line)
            new_term = re.match(capital_pattern, line)
            #print(line.split(",")[0])
            #print(line.split(",")[0] + " / " + term + " // " + new_term.group())
            if(new_term):
                #error: repeats some Terms but may be necessary for multiterm files
                # if(term != ""):
                #     i = 1
                    #parsed_data.append([ed, vol, page, term, definition, len(definition.split()), "NOT IMPLEMENTED", "NOT IMPLEMENTED", "NOT IMPLEMENTED"])
                #start new term    
                term = new_term.group().strip()
                definition += line.replace(term, "")
            #issue: what if term is two words?
            elif(line.split(',')[0].upper() == term):
                parsed_data.append([ed, vol, page, term, definition, len(definition.split()), "NOT IMPLEMENTED", "NOT IMPLEMENTED", "NOT IMPLEMENTED"])

                
                #same term but different definition    
                definition = "".join(line.split(",")[1:]).strip()
            else:
                #continue definition of current term bc it is a multiline definition
                definition += line
            
            #term = term[0] + term[1:].lower()
            #print(term + "!@# " + definition)
            
        #add last term in file
        parsed_data.append([ed, vol, page, term, definition, len(definition.split()), "NOT IMPLEMENTED", "NOT IMPLEMENTED", "NOT IMPLEMENTED"])
        
        
df = pd.DataFrame(parsed_data, columns=["edition", "volume", "page", "term", "definition", "numberOfWords", "numberOfPages", "relatedTerm", "topic/article"])

df.head(20)

,edition,volume,page,term,definition,numberOfWords,numberOfPages,relatedTerm,topic/article
0,7,2,1,A,. The first letter of the alphabet in every kn...,1571,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
1,7,2,2,AA,", a river of the province of Groningen, in the...",28,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
2,7,2,2,AA,a river in the province of Overyssel. in the N...,16,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
3,7,2,2,AA,a river of the province of Antwerp in the Neth...,17,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
4,7,2,2,AA,a river of France rising in the Pas de Calais;...,59,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
5,7,2,2,AA,a river in the Russian government of Courland ...,37,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
6,7,2,2,AA,a river in the Russian province of Livonia whi...,19,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
7,7,2,2,AAHUS,", a little town of Germany, in the circle of W...",41,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
8,7,2,2,AALBORG,", one of the four sees (stifts) into which the...",223,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
9,7,2,2,AALBORG,a city in Denmark the capital of the see of th...,167,NOT IMPLEMENTED,NOT IMPLEMENTED,NOT IMPLEMENTED
